<a href="https://colab.research.google.com/github/Hernanros/SOTA/blob/master/notebooks/labels/chrF_pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install wmd

     |████████████████████████████████| 112kB 2.7MB/s 
  Created wheel for wmd: filename=wmd-1.3.2-cp36-cp36m-linux_x86_64.whl size=629397 sha256=e7f52dec50a1e6425db2700d50bc5a30adc1d4847d8dc0303e34ce83d18503d7
  Stored in directory: /root/.cache/pip/wheels/a6/4d/7a/fc3fdbc12e33c0551301543e4441fb4783c809a30451387ffd
Successfully built wmd


In [4]:
!pip install wordfreq
!git clone https://github.com/brmson/dataset-sts
# !wget  http://nlp.stanford.edu/data/glove.840B.300d.zip

     |████████████████████████████████| 32.8MB 120kB/s 
     |████████████████████████████████| 4.9MB 41.5MB/s 
     |████████████████████████████████| 276kB 41.9MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.3.2-cp36-none-any.whl size=32817238 sha256=952504f69ffa3a22b406cae47ed4b25da31c75af1f836bb21acffbdf84311d63
  Stored in directory: /root/.cache/pip/wheels/8d/ba/84/ba6be76208bd2c2124b6586f7967fb87e9f9fb4b4827e5e2c9
  Created wheel for langcodes: filename=langcodes-2.0.0-cp36-none-any.whl size=5044047 sha256=7416de6f8f2b25e6b7d3c53b2974de25785b666f437834c5603fb75caa42bb94
  Stored in directory: /root/.cache/pip/wheels/c9/11/90/c7bba8118f3674d75e1457537635266a12538cf622a4684bb2
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=862305 sha256=c5cdf6df07c72e286e81fda74bfda0cf282a9ae902bc9d6ef6f2aa12a38abbf7
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built wor

In [5]:
% cd /content/drive/My Drive/NLP_Project

/content/drive/My Drive/NLP_Project


In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from wmd import WMD
import wordfreq
import pickle

In [7]:
zip_ref = zipfile.ZipFile("./glove.840B.300d.zip")
zip_ref.extractall("/tmp")
zip_ref.close()

In [9]:
glove_pd = pd.read_csv('glove_pd.csv',index_col=0)
#glove_pd = pd.read_csv(open('glove.840B.300d.txt'), sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in glove_pd.T.items()}
#glove_pd.to_csv('/content/drive/My Drive/NLP_Project/glove_pd.csv')
# np.savetxt(r'/content/drive/My Drive/NLP_Project/glove.txt', glove_pd.values, fmt='%d', delimiter=' ')

In [17]:
with open('combined_data.pickle', 'rb') as handle:
    df = pickle.load(handle)


In [21]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(df.iloc[9,0])
df['text_1_tokens'] =df.text_1.apply(lambda x:tokenizer.tokenize(x)) 
df['text_2_tokens'] =df.text_2.apply(lambda x:tokenizer.tokenize(x)) 

def embed (sent):
  toklist = tokenizer.tokenize(sent)
  embs = []
  freqs = []
  for word in toklist:
    try: 
      embs.append(glove[word.lower()])
      freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))
    except:
      try:
        embs.append(glove[word.strip("'")])
        freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))

      except:
        continue
  return embs, freqs

embed(df.iloc[9,0])
text_1_embeddings = df.text_1.apply(lambda x: embed(x) )
text_2_embeddings = df.text_2.apply(lambda x: embed(x) )

# ChrFScore

In [51]:
from nltk.translate import chrf_score
scores = []
err=0
for i in range(df.shape[0]):
  try:
    scores.append(chrf_score.sentence_chrf(df['text_1'][i],df['text_2'][i]))
  except:
    scores.append(0)
    err+=1
print( f"Num erros: {err}")

Num erros: 1


In [52]:
df['chrf_score'] = scores

In [54]:
OldMin = 0.0
OldMax = 1.0
NewMin = 0.0
NewMax = 5.0

new_scores = []
for score in scores:
    new_scores.append(((score - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin) + NewMin)


df['chrf_score_norm'] = new_scores

In [55]:
with open('combined_data.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)